# Определение категории новостей
Задача: с некоторых новостных сайтов были загружены тексты новостей за период несколько лет, причем каждая новость принаделжит к какой-то рубрике (science, style, culture, life, economics, business, travel, forces, media, sport). Нужно написать программу, которая автоматически определяет к какой рубрике можно отнести новость. 

В файле news_train.txt 60,000 строк, в каждой строке содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

sport 'tab' Сборная Канады по хоккею разгромила чехов 'tab' Сборная Канады по хоккею крупно об...

где 'tab' символ табуляции. В файле news_test.txt 15,000 строк, на каждой строке заголовок и новость без метки. Задача -- предсказать категорию всех новостей из тестового файла.

В качестве решения принимается файл из 15,000 строк, на каждой строке которого стоит метка, соответствующая одной из 10 категорий, пример такого файла в news_output_example.txt . Все файлы имеют кодировку utf-8.

In [1]:
import numpy as np
import pandas as pd
import re
categories=np.array([u'science', u'style', u'culture', u'life', u'economics', u'business', u'travel', u'forces', u'media', 
                     u'sport'])

In [2]:
def razbor_na_slova(f): #функция разбора предложений на слова
    f=f.lower()
    result_with_empty_words=re.split(u'[^а-я]', f)
    result=[]
    for i in result_with_empty_words:
        if i!=u'':
            result.append(i)
    return result

In [3]:
import codecs
file_sentences=codecs.open('news_train.txt', 'r', "utf-8") #считывание данных
strings = [] 
for line in file_sentences: 
    strings.append(line.strip().lower())

In [4]:
result=[]
y=[]
X_labels=[]
X_text=[]

for i in range(60000): #разбор считанных данных
    result = re.split('\t', strings[i] )
    y=np.append(y,np.where(categories==result[0])) #категория
    X_labels.append(razbor_na_slova(result[1])) #заголовки
    X_text.append(razbor_na_slova(result[2])) #текст
y=y.astype(int)

In [5]:
dict_all_words=dict()
# создание словаря 
for i in range(60000): 
    for j in X_labels[i]:
        if len(j)>2:
            if j not in dict_all_words:
                vect=[0,0,0,0,0,0,0,0,0,0]
            else:
                vect = dict_all_words.get(j)
            if y[i]==1 or y[i]==6: #нормирование
                slovo=8
            elif y[i]==5:
                slovo=4
            elif y[i]==7:
                slovo=2
            else:
                slovo=1                     
            vect[y[i]]+=slovo
            dict_all_words[j] = vect
    for j in X_text[i]:
        if len(j)>2:
            if j not in dict_all_words:
                vect=[0,0,0,0,0,0,0,0,0,0]
            else:
                vect = dict_all_words.get(j)
            if y[i]==1 or y[i]==6: #нормирование
                slovo=8
            elif y[i]==5:
                slovo=4
            elif y[i]==7:
                slovo=2
            else:
                slovo=1                    
            vect[y[i]]+=slovo
            dict_all_words[j] = vect

In [6]:
# подсчёт вероятностей
for i in dict_all_words:
    vect=list(dict_all_words.get(i))
    summa=sum(vect)
    for k in range(10):
        vect[k]/=float(summa)
    dict_all_words[i]=vect

In [7]:
#тестовый файл
file_sentences=codecs.open('news_test.txt', 'r', "utf-8")
strings_test = [] 
for line in file_sentences: 
    strings_test.append(line.strip().lower())
result=[]
X_labels_test=[]
X_text_test=[]

for i in range(15000):
    result = re.split('\t', strings_test[i] ) 
    X_labels_test.append(razbor_na_slova(result[0]))
    X_text_test.append(razbor_na_slova(result[1]))


In [8]:
#определение тем для теста
testing_dict=dict()
for i in range(15000):
    vect=[0,0,0,0,0,0,0,0,0,0]
    for j in X_labels_test[i]:
        if j in dict_all_words:
            for k in range(10):
                vect[k]+=dict_all_words.get(j)[k]
    for j in X_text_test[i]:
        if j in dict_all_words:
             for k in range(10):
                vect[k]+=dict_all_words.get(j)[k]
    testing_dict[i] = vect
    
y_test=[]
k=0
for i in testing_dict:
    y_test.append(testing_dict.get(i).index(max(testing_dict.get(i))))
    if testing_dict.get(i).index(max(testing_dict.get(i)))==0:
        k+=1

In [9]:
#запись в файл
answer=codecs.open('news_output.txt', 'w', "utf-8")
for i in range(15000):
    answer.write(categories[y_test[i]]+'\n')
answer.close()